In [78]:
# variables to edit!
# context = current plasmid with insert
# mutation_pos = the nucleotide right before the mutation
# addition = the mutation to add in

def calc_tm(primer, n):
    percent_gc = (primer.count('G') + primer.count('C')) / n
    return 81.5 + (41 * percent_gc) - (675 / n)

def reverse_primer(p):
    p = p.upper()
    rev_p = ''
    rev_d = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
    for nt in p:
        rev_p += rev_d[nt]
    return rev_p[::-1]

def count_gc(primer):
    return (primer.count('G') + primer.count('C')) / len(primer)

def get_fwd_primer_mut(context, mutation_pos, mut):

    best_seq = context[mutation_pos-15:mutation_pos] + mut + context[mutation_pos+3:mutation_pos+22]
    best_tm = calc_tm(best_seq, len(best_seq))
    gc_count = count_gc(best_seq)
    if best_tm >= 78. and best_seq[-1] in ('G', 'C') and best_seq[0] in ('G', 'C') and gc_count >= 0.4:
        return best_seq

    for neg in range(12,16):
        for pos in range(18,25):
            fwd = context[mutation_pos-neg:mutation_pos] + mut + context[mutation_pos+3:mutation_pos+pos]
            tm = calc_tm(fwd, len(fwd))
            gc_count = count_gc(fwd)
            if tm >= 78. and fwd[-1] in ('G', 'C') and fwd[0] in ('G', 'C') and gc_count >= 0.4:
                return fwd
            elif tm > best_tm:
                best_seq = fwd
                best_tm = tm
                best_gc = gc_count
    
    return(f'did not converge with Tm of {best_tm} and %GC of {best_gc}, but best seq is: {best_seq}')

def get_rev_primer_mut(context, mutation_pos, mut):

    best_seq = reverse_primer(context[mutation_pos-22:mutation_pos] + mut + context[mutation_pos+3:mutation_pos+15])
    best_tm = calc_tm(best_seq, len(best_seq))
    best_gc = count_gc(best_seq)
    if best_tm >= 78. and best_seq[-1] in ('G', 'C') and best_seq[0] in ('G', 'C') and best_gc >= 0.4:
        return best_seq

    for neg in range(18,25):
        for pos in range(12,16):
            rev = reverse_primer(context[mutation_pos-neg:mutation_pos] + mut + context[mutation_pos+3:mutation_pos+pos])
            tm = calc_tm(rev, len(rev))
            gc_count = count_gc(rev)
            if tm >= 78. and rev[-1] in ('G', 'C') and rev[0] in ('G', 'C') and gc_count >= 0.4:
                return rev
            elif tm > best_tm:
                best_seq = rev
                best_tm = tm
                best_gc = gc_count

    return(f'did not converge with Tm of {best_tm} and %GC of {best_gc}, but best Tm seq is: {best_seq}')

def align_dna(p, context):
    results = {k: 0 for k in range(len(context))}

    for idx in range(len(context)):
        count = 0
        for pidx, pnt in enumerate(p):
            if idx+pidx < len(context):
                if pnt == context[idx+pidx]:
                    count += 1
        results[idx] = count
    
    return max(results, key=lambda key: results[key])

def align_primers_to_context(fwd_p, rev_p, context, reverse=False):
    context = context.upper()

    fwd_pos = align_dna(fwd_p.upper(), context)
    rev_pos = align_dna(reverse_primer(rev_p.upper()), context)

    min_pos = min(fwd_pos, rev_pos)
    max_pos = max(fwd_pos+len(fwd_p), rev_pos+len(rev_p))

    print(f"5' {'-' * (fwd_pos-(min_pos-3))}>-{fwd_p}->{'-' * (3 + max_pos - (fwd_pos + len(fwd_p)))} 3' forward primer")
    fwd_match = ''
    for idx, nt in enumerate(fwd_p):
        if nt == context[idx+fwd_pos]:
            fwd_match += ' '
        else:
            fwd_match += 'X'
    print(f"   {' ' * (fwd_pos-(min_pos-5))}{fwd_match}{' ' * (5 + max_pos - (fwd_pos + len(fwd_p)))}   ")
    print(f"5' {context[min_pos-5:max_pos+5]} 3' context")
    print(f"3' {reverse_primer(context[min_pos-5:max_pos+5])[::-1]} 5' context")

    # rev_p and context are reversed again ([::-1]) b/c it's going 3' -> 5' in this visualization
    rev_match = ''
    for idx, nt in enumerate(rev_p[::-1]): 
        if nt == reverse_primer(context)[::-1][idx+rev_pos]:
            rev_match += ' '
        else:
            rev_match += 'X'
    print(f"   {' ' * (rev_pos-(min_pos-5))}{rev_match}{' ' * (5 + max_pos - (rev_pos + len(rev_p)))}   ")
    print(f"3' {'-' * (rev_pos-(min_pos-3))}<-{rev_p[::-1]}-<{'-' * (3 + max_pos - (rev_pos + len(rev_p)))} 5' reverse primer")

def aa_from_codon(codon):
    for k, v in ecoli_codon_dict.items():
        if codon in v:
            return k

ecoli_codon_dict = {
    'A': ['GCG', 'GCC', 'GCA', 'GCT'],
    'C': ['TGC', 'TGT'],
    'D': ['GAT', 'GAC'],
    'E': ['GAA', 'GAG'],
    'F': ['TTT', 'TTC'],
    'G': ['GGC', 'GGT', 'GGG', 'GGA'],
    'H': ['CAT', 'CAC'],
    'I': ['ATT', 'ATC', 'ATA'],
    'K': ['AAA', 'AAG'],
    'L': ['CTG', 'TTA', 'TTG', 'CTT', 'CTC', 'CTA'],
    'M': ['ATG'],
    'N': ['AAC', 'AAT'],
    'P': ['CCG', 'CCA', 'CCT', 'CCC'],
    'Q': ['CAG', 'CAA'],
    'R': ['CGT', 'CGC', 'CGG', 'CGA', 'AGA', 'AGG'], 
    'S': ['AGC', 'TCT', 'AGT', 'TCC', 'TCA', 'TCG'],
    'T': ['ACC', 'ACG', 'ACT', 'ACA'],
    'V': ['GTG', 'GTT', 'GTC', 'GTA'],
    'W': ['TGG'],
    'Y': ['TAT', 'TAC'],
    '*': ['TAA', 'TGA', 'TAG']
}

In [79]:
context='DNA_CONTEXT_SEQ_HERE'
aa_mut_pos = 1 # int
new_aa = 'T' # 1-letter aa code


fwd_p = get_fwd_primer_mut(context.upper().strip(), (aa_mut_pos-1)*3, ecoli_codon_dict[new_aa][0])
rev_p = get_rev_primer_mut(context.upper().strip(), (aa_mut_pos-1)*3, ecoli_codon_dict[new_aa][0])

print(fwd_p)
print(rev_p)
print('')

# check that things look correct 
align_primers_to_context(fwd_p, rev_p, context)

CACCGAACACATCACCTTTAACGTATCCTGC
GATACGTTAAAGGTGATGTGTTCGGTGTTGCC
5' -------->-CACCGAACACATCACCTTTAACGTATCCTGC->--- 3' forward primer
                          XXX                       
5' ATGTGGGCAACACCGAACACATCTTATTTAACGTATCCTGCCCAGT 3' context
3' TACACCCGTTGTGGCTTGTGTAGAATAAATTGCATAGGACGGGTCA 5' context
                          XXX                       
3' ---<-CCGTTGTGGCTTGTGTAGTGGAAATTGCATAG-<------- 5' reverse primer


In [ ]:
# batch
mutations = [
    (1, 'M'),
    (2, 'E'), # etc
]

name_idx = 1
for aa_mut_pos, new_aa in mutations:
    print(f'mut.{name_idx}')
    fwd_p = get_fwd_primer_mut(context.upper().strip(), (aa_mut_pos-1)*3, ecoli_codon_dict[new_aa][0])
    rev_p = get_rev_primer_mut(context.upper().strip(), (aa_mut_pos-1)*3, ecoli_codon_dict[new_aa][0])
    print(fwd_p)
    print(rev_p)
    print('')
    align_primers_to_context(fwd_p, rev_p, context)
    print('')
